In [1]:
from src.services.stt.stt import stt_from_audio
from src.services.assistant.assistant import Assistant

In [2]:
assistant = Assistant()

In [3]:
texto = stt_from_audio('audios/fibonnaci.wav')

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cpu
/opt/anaconda3/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


In [4]:
resposta = assistant.reply(texto)
print("EchoLoco:", resposta)

EchoLoco: Olá! A sequência de Fibonacci é uma série numérica onde cada número é a soma dos dois anteriores, começando geralmente com 0 e 1. A função recursiva para calcular o n-ésimo termo dessa sequência em Python é bem simples.

Aqui está um exemplo de como você pode implementá-la:

```python
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

# Exemplo de uso
n = 10  # Para encontrar o 10º termo
print(f"O {n}º termo da sequência de Fibonacci é: {fibonacci(n)}")
```

### Explicação:
1. **Base da Recursão**: A função verifica se n é 0 ou 1, retornando 0 ou 1 respectivamente. Essas são as condições de parada.
2. **Chamada Recursiva**: Se n for maior que 1, a função chamará a si mesma duas vezes: uma para (n-1) e outra para (n-2), somando os resultados.

### Considerações:
- Essa implementação é simples, mas não é a mais eficiente, pois recalcula valores repetidamente. Para casos maiores de n, 

In [6]:
texto

'Olá, eu queria muito uma função recursiva que faz Fibonat em Python.'

In [9]:
from src.services.tts.tts import load_tts_pipeline, tts_from_text

tts_pipe, language, output_dir, audio_format = load_tts_pipeline()
audio_path = tts_from_text(
    "vai tomar no cu Daniel filho da puta",
    tts_pipe,
    language=language,
    output_dir=output_dir,
    file_name="tts_output",
    audio_format=audio_format
)
print(f"Áudio gerado em: {audio_path}")

Áudio gerado em: ./audios_sintetizados/tts_output.wav


In [10]:
from IPython.display import Audio

Audio(audio_path)
